In [2]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions_table_for_gas_costs import *

deployers_df, chainlink_keepers_df, service_accounts_df = fetch_systems_df()

In [30]:
service_accounts_df = service_accounts_df[service_accounts_df["chain_id"] == 1]
deployers_df = deployers_df[deployers_df["chain_id"] == 1]
address_to_name = service_accounts_df.set_index("address")["name"].to_dict()
address_to_name[deployers_df["deployer"].values[0]] = "deployer"


address_to_type = service_accounts_df.set_index("address")["type"].to_dict()
address_to_type[deployers_df["deployer"].values[0]] = "deployer"

address_to_type

{'0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c': 'pricing',
 '0x241b8f1fA50F1Ce8fb78e7824757280feEb2aea3': 'debtReporting',
 '0xB17b69B4Cbb288aD2Caee9De2d8C3AD3C80017e6': 'swaps',
 '0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72': 'rewards',
 '0xa1c56731e180D7F43d06B9f65CecfE7A2F724dBE': 'stats',
 '0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245': 'solver',
 '0x54baDF9b444bd38a2E3909287D7E4E954778AeF2': 'solver',
 '0x3D1f51c23d1586c062B4bECa120bfCAf064e0cdC': 'solver',
 '0x6ec140890269992964d98DAe4De023181F4eEAF7': 'solver',
 '0x187dD9E0BE80972Ab2D14b5A5F69C1f5540Ace57': 'solver',
 '0x0460aE025d45C570D40c369e06e943cEB931Ac10': 'solver',
 '0xA83B79408D8DfC06DFfbBeA74a39EbD2769fb6e2': 'stats',
 '0x6b9d01f42A4e93aEBD412C24D6f3B949EEF500BE': 'stats',
 '0x923720b4aA90CdEc2c241682F89e6bF02FeBD509': 'stoke',
 '0x44a427518a1AFD356dab0f4c521331F2c8771FA7': 'stoke',
 '0x3096a687bAFB44D81D294450Da4B1cf9a38E401A': 'rewards',
 '0x77601882d70B3A391d2be4204c386d91037aC806': 'rewards',
 '0x123cC4AFA59160C6328C01

In [4]:
from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df
from mainnet_launch.database.schema.full import Transactions, Blocks
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

# chainlink_keepers_df['from_address'].tolist() +
full_tx_df = merge_tables_as_df(
    [
        TableSelector(
            Transactions,
            # filter to only your deployers + service accounts
            row_filter=Transactions.from_address.in_(
                deployers_df["deployer"].tolist() + service_accounts_df["address"].tolist()
            ),
        ),
        TableSelector(Blocks, select_fields=Blocks.datetime, join_on=Transactions.block == Blocks.block),
    ],
    where_clause=Blocks.chain_id == 1,
)

In [65]:
from decimal import Decimal, getcontext

# 1) set precision high enough for 18 decimal places
getcontext().prec = 28

# 2) define a Decimal constant for the wei→ETH divisor
WEI_IN_ETH = Decimal(10) ** 18

# 3) compute exact cost per row
full_tx_df["precise_gas_cost_in_eth"] = full_tx_df.apply(
    lambda row: (Decimal(row["gas_used"]) * Decimal(row["effective_gas_price"])) / WEI_IN_ETH, axis=1
)
full_tx_df["precise_gas_cost_in_eth"] = full_tx_df["precise_gas_cost_in_eth"].astype(float)
full_tx_df["label"] = full_tx_df["from_address"].map(address_to_name)
full_tx_df["type"] = full_tx_df["from_address"].map(address_to_type)
full_tx_df["hour"] = full_tx_df["datetime"].dt.hour
full_tx_df

,tx_hash,block,chain_id,from_address,to_address,effective_gas_price,gas_used,gas_cost_in_eth,datetime,precise_gas_cost_in_eth,label,type,hour
0,0x2c4950ee4d5b52f399284535b5fdf2e16aa3884930f4...,22373242,1,0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,2338262960,3196367,0.007474,2025-04-29 07:31:47+00:00,0.007474,Solver Exec (autoETH),solver,7
1,0xc32195bb75b6248ca0324b18ffc192c41f843f6809cf...,21977420,1,0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,2586548338,5334383,0.013798,2025-03-05 01:31:23+00:00,0.013798,Solver Exec (autoETH),solver,1
2,0xe18cb5060a7fdf25de7c2a10dc70f8b4c6bd8e97b173...,22295306,1,0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,2373350044,2656601,0.006305,2025-04-18 10:31:47+00:00,0.006305,Solver Exec (autoETH),solver,10
3,0x47160321b1d89999ee5d8ea677e5d25eab34c3c428ba...,21318711,1,0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,20903917740,3145556,0.065754,2024-12-03 01:31:23+00:00,0.065754,Solver Exec (autoETH),solver,1
4,0xea6e185ee58d279ceef5a4d95e7387e0a6c32e711d6d...,20910416,1,0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,10854184811,2942835,0.031942,2024-10-07 01:32:35+00:00,0.031942,Solver Exec (autoETH),solver,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33818,0x0423636f3cb24cbe6988ef9db8bdbc4448a9969f8047...,22907848,1,0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c,0x53ff9D648a8A1cf70c6B60ae26B93047cc24066f,2022078818,40641,0.000082,2025-07-13 04:23:59+00:00,0.000082,Custom Oracle Executor,pricing,4
33819,0xbe72b5a8644483031bd7f03da1addd626bec609fe3b0...,22897970,1,0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c,0x67c15923248baf96FB3944A2A326a237B4c21030,4732406951,128453,0.000608,2025-07-11 19:18:59+00:00,0.000608,Custom Oracle Executor,pricing,19
33820,0x72467e4156dc1a9a4bef005a0aa48098f108ab1e7e19...,22902880,1,0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c,0x53ff9D648a8A1cf70c6B60ae26B93047cc24066f,2075522012,40629,0.000084,2025-07-12 11:46:59+00:00,0.000084,Custom Oracle Executor,pricing,11
33821,0xb98ccc75c51539097a2786a4da1c5db68f4f1bfa1d84...,22899543,1,0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c,0x53ff9D648a8A1cf70c6B60ae26B93047cc24066f,2634963050,40629,0.000107,2025-07-12 00:34:59+00:00,0.000107,Custom Oracle Executor,pricing,0


In [54]:
type_df = (
    full_tx_df.groupby(["type", "datetime"])["gas_cost_in_eth"]
    .sum()
    .reset_index()
    .pivot(index="datetime", columns="type", values="gas_cost_in_eth")
    .fillna(0)
    .resample("1D")
    .sum()
    .rolling(window=7, min_periods=1)
    .sum()
)  # 7-day window, requires at least 1 day to start

fig = px.bar(
    type_df,
    title="7-day rolling sum Mainnet Tokemak's EOA gas costs",
)
fig.update_yaxes(title_text="ETH")
fig.show()

In [84]:
address_to_type["0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72"]

'rewards'

In [85]:
type_df = (
    full_tx_df.groupby(["type", "datetime"])["gas_cost_in_eth"]
    .sum()
    .reset_index()
    .pivot(index="datetime", columns="type", values="gas_cost_in_eth")
    .fillna(0)
    .resample("1D")
    .sum()
    .rolling(window=30, min_periods=1)
    .sum()
)  # 7-day window, requires at least 1 day to start

fig = px.bar(
    type_df,
    title="30-day rolling sum Mainnet Tokemak's EOA gas costs",
)
fig.update_yaxes(title_text="ETH")
fig.show()

In [82]:
df2 = (
    full_tx_df.groupby(["label", "datetime"])["precise_gas_cost_in_eth"]
    .sum()
    .reset_index()
    .pivot(index="datetime", columns="label", values="precise_gas_cost_in_eth")
    .fillna(0)
    .resample("1D")
    .sum()
)
fig = px.bar(
    df2,
    title="Cumulative Mainnet Tokemak's EOA Precise gas costs",
)
fig.update_yaxes(title_text="ETH")
fig.show()

In [73]:
Liquidator_df = full_tx_df[full_tx_df["label"] == "Liquidator"].copy()


px.bar(
    Liquidator_df.groupby("hour")["block"].count(), title="Count of all time Liquidiator Transactions by hour"
).update_yaxes(title_text="Count").show()

In [81]:
Liquidator_df["effective_gas_price_gwei"] = Liquidator_df["effective_gas_price"] / 1e9

fig = px.box(
    Liquidator_df,
    x="hour",
    y="effective_gas_price_gwei",
    # points='all',            # show all underlying points
    title="Distribution of Effective Gas Price by Hour",
    labels={"hour": "Hour of Day", "effective_gas_price_gwei": "Effective Gas Price (Gwei)"},
)
fig.update_layout(
    yaxis_title="Effective Gas Price (Gwei)",
    xaxis_title="Hour of Day",
    boxmode="group",  # ensures boxes don’t overlap if you add more traces
)
fig.show()

In [76]:
px.bar(
    Liquidator_df.groupby("hour")["effective_gas_price"].mean(),
    title="Count of all time Liquidiator Transactions by hour",
).update_yaxes(title_text="Count").show()

In [ ]:
# I want some representi

In [62]:
address_to_name

{'0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c': 'Custom Oracle Executor',
 '0x241b8f1fA50F1Ce8fb78e7824757280feEb2aea3': 'Debt Reporting',
 '0xB17b69B4Cbb288aD2Caee9De2d8C3AD3C80017e6': 'Swap Route Watcher',
 '0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72': 'Liquidator',
 '0xa1c56731e180D7F43d06B9f65CecfE7A2F724dBE': 'Points Boost',
 '0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245': 'Solver Exec (autoETH)',
 '0x54baDF9b444bd38a2E3909287D7E4E954778AeF2': 'Solver Exec (autoLRT)',
 '0x3D1f51c23d1586c062B4bECa120bfCAf064e0cdC': 'Solver Exec (balETH)',
 '0x6ec140890269992964d98DAe4De023181F4eEAF7': 'Solver Exec (dineroETH)',
 '0x187dD9E0BE80972Ab2D14b5A5F69C1f5540Ace57': 'Solver Exec (autoUSD)',
 '0x0460aE025d45C570D40c369e06e943cEB931Ac10': 'Solver Exec (autoDOLA)',
 '0xA83B79408D8DfC06DFfbBeA74a39EbD2769fb6e2': 'Incentive Harvester',
 '0x6b9d01f42A4e93aEBD412C24D6f3B949EEF500BE': 'Keeper Executor',
 '0x923720b4aA90CdEc2c241682F89e6bF02FeBD509': 'Auto Rewards (Signer/RewardHash)',
 '0x44a427518a1A

In [20]:
px.scatter(full_tx_df.set_index("datetime").resample("1d")["effective_gas_price"].agg(["median", "mean"]))

In [10]:
df.tail(1).T.round(4).sort_values("2025-07-14 00:00:00+00:00", ascending=False)

datetime,2025-07-14 00:00:00+00:00
label,
Liquidator,30.6808
Debt Reporting,29.8371
Solver Exec (autoETH),16.0058
Solver Exec (balETH),14.1719
Solver Exec (autoLRT),11.0170
Custom Oracle Executor,8.5608
deployer,8.0252
Solver Exec (autoUSD),6.6594
Incentive Harvester,6.4438


In [23]:
address_to_name

{'0x1b9841A65c6777fdE03Be97C9A9E70C3d5C01E9c': 'Custom Oracle Executor',
 '0x241b8f1fA50F1Ce8fb78e7824757280feEb2aea3': 'Debt Reporting',
 '0xB17b69B4Cbb288aD2Caee9De2d8C3AD3C80017e6': 'Swap Route Watcher',
 '0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72': 'Liquidator',
 '0xa1c56731e180D7F43d06B9f65CecfE7A2F724dBE': 'Points Boost',
 '0xE08cC4Bc6BD77774fE107f7Ffb48adD75Ae53245': 'Solver Exec (autoETH)',
 '0x54baDF9b444bd38a2E3909287D7E4E954778AeF2': 'Solver Exec (autoLRT)',
 '0x3D1f51c23d1586c062B4bECa120bfCAf064e0cdC': 'Solver Exec (balETH)',
 '0x6ec140890269992964d98DAe4De023181F4eEAF7': 'Solver Exec (dineroETH)',
 '0x187dD9E0BE80972Ab2D14b5A5F69C1f5540Ace57': 'Solver Exec (autoUSD)',
 '0x0460aE025d45C570D40c369e06e943cEB931Ac10': 'Solver Exec (autoDOLA)',
 '0xA83B79408D8DfC06DFfbBeA74a39EbD2769fb6e2': 'Incentive Harvester',
 '0x6b9d01f42A4e93aEBD412C24D6f3B949EEF500BE': 'Keeper Executor',
 '0x923720b4aA90CdEc2c241682F89e6bF02FeBD509': 'Auto Rewards (Signer/RewardHash)',
 '0x44a427518a1A

In [25]:
df.tail(1).T.round(4).sort_values("2025-07-14 00:00:00+00:00", ascending=False)

datetime,2025-07-14 00:00:00+00:00
label,
Liquidator,30.6808
Debt Reporting,29.8371
Solver Exec (autoETH),16.0058
Solver Exec (balETH),14.1719
Solver Exec (autoLRT),11.0170
Custom Oracle Executor,8.5608
deployer,8.0252
Solver Exec (autoUSD),6.6594
Incentive Harvester,6.4438


In [ ]:
# ok, this is broken in some imporant way

# https://etherscan.io/address/0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72#analytics

# etherscan is showing 30.643 ETH
# I am showing 30.6808

# i have 2,622 transactions
# etherscan has 2,662 transactions

# So etherscna sees 40 more transactions than I do but their gas costs are lower

: 

: 

: 

: 

In [ ]:
# # very close, but not exactly the same as off by a bit 30.643 (is what they say),
# https://etherscan.io/address/0x0b1EAA1CF011C80f075958Cf5B6bD49Abc3D7a72#analytics

: 

: 

In [ ]:
address_to_name

: 

: 

In [ ]:
full_tx_df[full_tx_df["label"] == "Solver Exec (autoDOLA)"]

: 

: 

In [ ]:
full_tx_df["label"].value_counts()

: 

: 

In [ ]:
deployers_df

: 

: 

: 

: 